In [ ]:
import numpy as np
from scipy import linalg
import scipy.sparse as sp

import sys
from pathlib import Path


negf_path = Path('..').resolve()
sys.path.insert(0, str(negf_path))


from device import Device
from hamiltonian import Hamiltonian

from typing import Tuple

def GzerozeroH_W_sparse(wmH: sp.spmatrix, t: sp.spmatrix) -> np.ndarray:
    """
    Surface Green's function calculation optimized for sparse matrices.
    """
    N = wmH.shape[0]
    

    wmH_dense = wmH.toarray()
    t_dense = t.toarray()

    A = np.zeros((2*N, 2*N), dtype=complex)
    B = np.zeros((2*N, 2*N), dtype=complex)
    
    A[:N, N:2*N] = np.eye(N)
    A[N:2*N, :N] = -t_dense.conj().T
    A[N:2*N, N:2*N] = wmH_dense
    
    B[:N, :N] = np.eye(N)
    B[N:2*N, N:2*N] = t_dense
    

    try:
        eigenvalues, eigenvectors = linalg.eig(A, B)
    except linalg.LinAlgError:
        print("Warning: Using pseudo-inverse for eigenvalue problem")
        B_pinv = linalg.pinv(B)
        eigenvalues, eigenvectors = linalg.eig(B_pinv @ A)
    

    magnitudes = np.abs(eigenvalues)

    valid_mask = np.isfinite(magnitudes) & (magnitudes > 1e-12)
    valid_eigenvalues = eigenvalues[valid_mask]
    valid_eigenvectors = eigenvectors[:, valid_mask]
    valid_magnitudes = magnitudes[valid_mask]
    
    if len(valid_eigenvalues) == 0:
        raise ValueError("No valid eigenvalues found")
    

    real_parts = np.real(valid_eigenvalues)
    sorted_indices = np.lexsort((valid_magnitudes, real_parts))
    
 
    sorted_eigenvectors = valid_eigenvectors[:, sorted_indices]
    
    Z11 = sorted_eigenvectors[:N, :N]
    Z21 = sorted_eigenvectors[N:2*N, :N]
    

    Z11_inv = linalg.pinv(Z11, rtol=1e-12)
    Gzeta = Z21 @ Z11_inv

    
    return Gzeta
def self_energy(side, E, ky, ham: Hamiltonian) -> np.ndarray:
    """
    Corrected self-energy calculation.
    """
    H00, H01, H10 = ham.get_H00_H01_H10(ky, side=side, sparse=True)
    
    N = H00.shape[0]
    eta = 1e-6
    wmH = E * sp.eye(N, dtype=complex) - H00 + 1j * eta * sp.eye(N, dtype=complex)
    
    if side == "left":
        
        Gzeta = GzerozeroH_W_sparse(wmH, H10)

        selfenergy = H10.toarray() @ Gzeta @ H10.conj().T.toarray()
    else:
        Gzeta = GzerozeroH_W_sparse(wmH, H01)
        selfenergy = H01.toarray() @ Gzeta @ H01.conj().T.toarray()
    
    return selfenergy
    
    
def calculate_self_energy_nanowire(H00: sp.spmatrix, H01: sp.spmatrix, 
                                  E: float, eta: float = 1e-6) -> np.ndarray:
    """
    Calculate self-energy for silicon nanowire system.
    
    Parameters:
    -----------
    H00 : sp.spmatrix
        Unit cell Hamiltonian (sparse)
    H01 : sp.spmatrix
        Coupling between unit cells (sparse)
    E : float
        Energy
    eta : float
        Small imaginary part
        
    Returns:
    --------
    np.ndarray
        Self-energy matrix
    """
    N = H00.shape[0]
    
    wmH = E * sp.eye(N, dtype=complex) - H00 + 1j * eta * sp.eye(N, dtype=complex)
    
    print(f"Energy E = {E}")
    print(f"Matrix size N = {N}")
    print(f"wmH shape: {wmH.shape}")
    print(f"H01 shape: {H01.shape}")
    
    wmH_dense = wmH.toarray()
    H01_dense = H01.toarray()
    
    print(f"wmH trace: {np.trace(wmH_dense):.6f}")
    print(f"H01 trace: {np.trace(H01_dense):.6f}")
    print(f"wmH norm: {np.linalg.norm(wmH_dense):.6f}")
    print(f"H01 norm: {np.linalg.norm(H01_dense):.6f}")
    
    if np.allclose(wmH_dense, 0):
        print("WARNING: wmH is zero!")
        return np.zeros((N, N), dtype=complex)
    
    if np.allclose(H01_dense, 0):
        print("WARNING: H01 is zero!")
        return np.zeros((N, N), dtype=complex)
    
    Gzeta = GzerozeroH_W_sparse(wmH, H01)
    selfenergy = H01_dense @ Gzeta @ H01_dense.conj().T
    
    print(f"Self-energy trace: {np.trace(selfenergy):.6f}")
    print(f"Self-energy norm: {np.linalg.norm(selfenergy):.6f}")
    
    return selfenergy



In [6]:
dev = Device(channel_length=2.15e-9, channel_thickness=1.5e-9)
ham = Hamiltonian(dev)
sr = self_energy("right", 100, 0, ham=ham)
sl = self_energy("left", 100, 0, ham=ham)

In [7]:
sr[-60:,-60:]

array([[-4.55507707e-02+1.22917114e-09j, -8.12514359e-02-8.92752151e-09j,
         6.93670929e-03-1.56554688e-09j, ...,
         7.49027248e-07-5.98983230e-09j, -1.77965880e-06+1.46797472e-09j,
         6.20722197e-06-5.03705571e-08j],
       [ 7.81353489e-02+4.33167568e-10j,  1.68954266e-01-8.98918686e-09j,
        -2.04229042e-02-6.37950166e-10j, ...,
         7.72400715e-06-4.19769477e-09j, -6.98086866e-07+7.65388693e-10j,
         1.18740009e-05-4.30366483e-08j],
       [-7.74860548e-03+3.37825909e-09j, -2.23787092e-02-1.68203455e-08j,
         6.86162251e-03-3.34853735e-09j, ...,
        -4.45340220e-06-1.63663893e-08j,  2.14115758e-06+4.12371913e-09j,
        -1.44226100e-05-1.25189205e-07j],
       ...,
       [-1.02791599e-04+1.37485242e-09j, -1.73961173e-04-8.40821261e-10j,
         9.38622380e-05-9.49860421e-10j, ...,
        -1.99647164e-02-1.09055953e-09j,  3.76345151e-03+5.09031541e-10j,
        -3.91584421e-02-9.72792652e-09j],
       [-5.66073412e-05+4.59641258e-09j, -6.

In [8]:
sl

array([[-0.01221328+1.05948033e-09j,  0.01265503-9.69651102e-09j,
         0.01870528-5.61859056e-10j, ...,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
       [-0.02180583+1.20123333e-08j,  0.03837269-4.10920642e-09j,
         0.03341736-1.70492240e-08j, ...,  0.        +0.00000000e+00j,
        -0.        +0.00000000e+00j, -0.        +0.00000000e+00j],
       [-0.01934147-1.44686937e-08j,  0.03427261+1.94628488e-08j,
         0.02347007+1.84878292e-08j, ...,  0.        +0.00000000e+00j,
        -0.        +0.00000000e+00j, -0.        +0.00000000e+00j],
       ...,
       [ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j, ...,  0.        +0.00000000e+00j,
        -0.        +0.00000000e+00j, -0.        +0.00000000e+00j],
       [ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j, ...,  0.        +0.00000000e+00j,
        -0.        +0.00000000e+00j